In [1]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM

import nltk
from nltk import word_tokenize
nltk.download('punkt')

import numpy as np

[nltk_data] Downloading package punkt to C:\Users\Oscar
[nltk_data]     Pang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# model
loaded_model = keras.models.load_model('reviews.model')

# dataset
imdb = keras.datasets.imdb

# word to index
word2index = imdb.get_word_index()

# variables
words = 20000
max_len = 100

In [3]:
# load IMDB dataset as lists of integers
words = 20000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=words)

# pad the arrays so they all have the same length
train_data = keras.preprocessing.sequence.pad_sequences(train_data, maxlen=max_len)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, maxlen=max_len)

In [7]:
# Create a model
embedding_dimension = 16
model = Sequential()

# The shape is: (samples, vocab_size, embedding_dimension)
model.add(Embedding(words, embedding_dimension, input_length=max_len))

# Add 2 bidirectional LSTMS
model.add(keras.layers.Bidirectional(LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.5)))
model.add(keras.layers.Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.5)))

# Add a classifier on top
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Train the model
model.fit(
    train_data,
    train_labels,
    epochs=5,
    batch_size=32,
    validation_split=0.2
)

model.save("reviews.model")

Epoch 1/5
625/625 [==============================] - 180s 279ms/step - loss: 0.4588 - acc: 0.7643 - val_loss: 0.3408 - val_acc: 0.8590
Epoch 2/5
625/625 [==============================] - 174s 279ms/step - loss: 0.2684 - acc: 0.8924 - val_loss: 0.3619 - val_acc: 0.8532
Epoch 3/5
625/625 [==============================] - 171s 273ms/step - loss: 0.1954 - acc: 0.9268 - val_loss: 0.3963 - val_acc: 0.8454
Epoch 4/5
625/625 [==============================] - 171s 274ms/step - loss: 0.1539 - acc: 0.9436 - val_loss: 0.4148 - val_acc: 0.8402
Epoch 5/5
625/625 [==============================] - 169s 270ms/step - loss: 0.1198 - acc: 0.9556 - val_loss: 0.5057 - val_acc: 0.8410
INFO:tensorflow:Assets written to: reviews.model\assets


In [10]:
sentence = input("Enter a movie review")
print(sentence)

# tokenize the sentence
tokened = word_tokenize(sentence)
test = []

# for each word in the sentence, get the corresponding index
for word in tokened:
    if (word not in word2index):
      continue
    if (word2index[word] > words):
      continue
    test.append(word2index[word] + 3)

# pad sequences taking consideration of the max length
test = keras.preprocessing.sequence.pad_sequences([test], maxlen=max_len) 

predicted = loaded_model.predict(test)
print(predicted)

if (predicted > 0.5):
    print("Positive")
else:
    print("Negative")

never seen anything worse than this
1/1 [==============================] - 0s 41ms/step
[[0.01792799]]
Negative
